In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Configuração do ambiente para utilização do Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Leitura dos dados

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/installments_payments.csv')
df1.head()

SK_ID_PREV  SK_ID_CURR  NUM_INSTALMENT_VERSION  NUM_INSTALMENT_NUMBER  \
0     1054186      161674                     1.0                      6   
1     1330831      151639                     0.0                     34   
2     2085231      193053                     2.0                      1   
3     2452527      199697                     1.0                      3   
4     2714724      167756                     1.0                      2   

   DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  AMT_PAYMENT  
0          -1180.0             -1187.0        6948.360     6948.360  
1          -2156.0             -2156.0        1716.525     1716.525  
2            -63.0               -63.0       25425.000    25425.000  
3          -2418.0             -2426.0       24350.130    24350.130  
4          -1383.0             -1366.0        2165.040     2160.585

In [ ]:
# Lê o arquivo Parquet
dados = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/installments_payments.csv",header=True)

# Mostra os dados
dados.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|   1054186|    161674|                   1.0|                    6|        -1180.0|           -1187.0|       6948.36|    6948.36|
|   1330831|    151639|                   0.0|                   34|        -2156.0|           -2156.0|      1716.525|   1716.525|
|   2085231|    193053|                   2.0|                    1|          -63.0|             -63.0|       25425.0|    25425.0|
|   2452527|    199697|                   1.0|                    3|        -2418.0|           -2426.0|      24350.13|   24350.13|
|   2714724|    167756|                   1.0|                    2|        -1383.0

## Criação de flags para nos auxiliar na visão temporal dos dados

Nesta etapa, foi executada uma consulta SQL que adiciona novas colunas ao conjunto de dados. Essas colunas funcionam como marcadores temporais (ou flags), indicando se cada registro pertence aos últimos 3, 6, 12, 24 ou 36 meses, com base na variável DAYS_INSTALMENT. Para isso, foram utilizados intervalos de dias equivalentes a cada período (por exemplo, -90 dias para 3 meses, -180 dias para 6 meses, e assim por diante). Assim, cada linha do DataFrame recebe o valor 1 quando pertence ao período correspondente e 0 caso contrário.

Essa transformação é útil porque permite filtrar e analisar comportamentos recentes de forma dinâmica, facilitando comparações entre diferentes janelas de tempo. Por fim, o resultado foi ordenado pela coluna SK_ID_PREV e salvo em uma nova visão temporária chamada df_temp_01, que servirá de base para as próximas etapas de análise.

In [ ]:
## Habilitando uso do SparkSQL
dados.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN DAYS_INSTALMENT >= -90 THEN 1
        ELSE 0
    END AS ultimos_3_meses,
    CASE
        WHEN DAYS_INSTALMENT >= -180 THEN 1
        ELSE 0
    END AS ultimos_6_meses,
    CASE
        WHEN DAYS_INSTALMENT >= -360 THEN 1
        ELSE 0
    END AS ultimos_12_meses,
    CASE
        WHEN DAYS_INSTALMENT >= -720 THEN 1
        ELSE 0
    END AS ultimos_24_meses,
    CASE
        WHEN DAYS_INSTALMENT >= -1080 THEN 1
        ELSE 0
    END AS ultimos_36_meses
FROM dados
ORDER BY `SK_ID_PREV`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show()

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+---------------+---------------+----------------+----------------+----------------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|ultimos_3_meses|ultimos_6_meses|ultimos_12_meses|ultimos_24_meses|ultimos_36_meses|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+---------------+---------------+----------------+----------------+----------------+
|   1000001|    158271|                   2.0|                    2|         -238.0|            -244.0|     62039.115|  62039.115|              0|              0|               1|               1|               1|
|   1000001|    158271|                   1.0|                    1|         -268.0|            -294.0|       6404.31|    6404.31|              

## Sumarizar na visão cliente (Automatizada)

Nesta etapa, os dados foram resumidos por cliente de forma automatizada, agrupando as informações por contrato (SK_ID_PREV) e calculando estatísticas para diferentes períodos de tempo. Foram utilizadas as colunas de marcação temporal criadas anteriormente, que indicam se cada registro pertence aos últimos 3, 6, 12, 24 ou 36 meses. Para cada uma dessas janelas, o código calcula métricas como soma, média, máximo e mínimo das variáveis numéricas, considerando apenas os registros dentro do período correspondente. No caso das colunas relacionadas a dias, são calculados apenas os valores máximo e mínimo. O resultado é um novo DataFrame (df_temp_02) que traz um resumo consolidado de cada cliente, facilitando análises e comparações de comportamento ao longo do tempo.

In [ ]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when

# Definir as colunas para agregação
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')

colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

expressoes_agregacao = []

for flag in colunas_flags:
  for coluna in colunas_agregacao_total:
    if 'DAY' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_INSTALLMENTS"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_INSTALLMENTS"))
    else:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_INSTALLMENTS"))
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_INSTALLMENTS"))
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_INSTALLMENTS"))
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_INSTALLMENTS"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_02 = df_temp_01.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# Mostrar o DataFrame resultante
df_temp_02.show()

+----------+----------------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+---------------------------------------------------------+---------------------------------------------------------+---------------------------------------------------------+---------------------------------------------------------+---------------------------------------------------+---------------------------------------------------+------------------------------------------------------+------------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+-----------------------------------------------+-----------------------------------------------+------

## Salvar a tabela sumarizada

Nesta etapa, o DataFrame df_temp_02 foi salvo em um único arquivo CSV chamado installments_agg.csv. O comando repartition(1) garante que o Spark gere apenas um arquivo, e o método write.mode("overwrite") sobrescreve arquivos anteriores, incluindo o cabeçalho. Essa etapa exporta o resultado final consolidado para uso em outras análises.

In [ ]:
df_temp_02 = df_temp_02.repartition(1)
df_temp_02.write.mode("overwrite").csv("installments_agg.csv",header=True)